Contexte: L'objectif de cet exercice est de prédire la fin d'un match league of legends, en fonction des dix premières minutes. 

Résultat: Le modèle prédit correctement dans 72% des cas.

Context: We're trying to guess how likely a game is to be won, based on the first 10 minutes of game. 

Result: We get an accuracy of 72%, this means that 30% of the time, players have an opportunity to make a comeback.  

In [21]:
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
from tensorflow.keras import layers

In [31]:
data = pd.read_csv("../input/league-of-legends-diamond-ranked-games-10-min/high_diamond_ranked_10min.csv")

# find nb of columns + number of rows of the data

print(data.shape)

# print the first rows of the data

data.head()

In [23]:
data.drop("gameId",axis=1, inplace=True)

In [24]:
data = data.sample(frac=1).reset_index(drop=True)

In [25]:
y = data['blueWins']

In [26]:
X = data.drop("blueWins", axis=1, inplace=False)

In [27]:
y.head()

In [29]:
#Here we heatmap the separated dataset to understand the correlations

sns.heatmap(X.corr())

In [36]:
# we'll train on 70% of the values
train_test_split = 0.7

# Here we want a measure of many exmaples we have, since X.shape returns a tuple, we can use this line to take the first number
num_examples = X.shape[0]

num_train_examples = int(np.floor(num_examples*train_test_split))

num_test_examples = int(np.ceil(num_examples - num_train_examples))

print(num_examples)
print(num_train_examples)
print(num_test_examples)

We come up with an optimizer function 

In [39]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001,beta_1=0.9,beta_2=0.999,epsilon=1e-07,amsgrad=False,name='Adam')

In [37]:
# here what matters is 38, the nb of features
inputs = tf.keras.Input(shape=(38,))

# here what matters is 16
x = tf.keras.layers.Dense(16, activation=tf.nn.relu)(inputs)
x = tf.keras.layers.Dense(16, activation=tf.nn.relu)(x)

# here what matters is 2, bc you want to predict either a blue victory or a red victory
outputs = tf.keras.layers.Dense(2, activation=tf.nn.softmax)(x)


model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.summary()

In [45]:
# now we compile the model 

model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [49]:
X_train = X.iloc[0:num_train_examples, :]
y_train = y.iloc[0:num_train_examples]

X_test = X.iloc[num_train_examples:num_examples, :]
y_test = y.iloc[num_train_examples:num_examples]

In [50]:
BATCH_SIZE=32
EPOCHS=300 

In [52]:
model.fit(
    x=X_train,
    y=y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    verbose=1,
    validation_split=0.2,
    shuffle=True
)

In [55]:
# does the model generalize to new data?

loss, accuracy = model.evaluate(X_test, y_test)